In [1]:
import numpy as np
import tensorflow as tf
import queue
import pickle
import zipfile
import collections
import math
import time

from collections import Counter

C:\Users\Angel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from train_util import *
from preprocessing import *
from layer import *
from model import *

In [3]:
test_word2vec()

Filling Unigram Table
Testing Model :skipgram, loss : Negative_Sampling
idx: 200, loss: 2.052836, time: 0.080785, 
idx: 200, loss: 1.903186, time: 0.065862, 
idx: 200, loss: 1.669679, time: 0.074799, 
idx: 200, loss: 1.480582, time: 0.063822, 
idx: 200, loss: 1.454608, time: 0.062802, 
idx: 200, loss: 1.470768, time: 0.061833, 
idx: 200, loss: 1.480899, time: 0.071850, 
idx: 200, loss: 1.412037, time: 0.063824, 
idx: 200, loss: 1.470574, time: 0.065792, 
idx: 200, loss: 1.453964, time: 0.071805, 
Testing Model :skipgram, loss : Softmax
idx: 200, loss: 2.078240, time: 0.093749, 
idx: 200, loss: 2.046164, time: 0.074830, 
idx: 200, loss: 1.987724, time: 0.064831, 
idx: 200, loss: 1.920189, time: 0.065791, 
idx: 200, loss: 1.859566, time: 0.078820, 
idx: 200, loss: 1.709318, time: 0.074806, 
idx: 200, loss: 1.630252, time: 0.083766, 
idx: 200, loss: 1.575712, time: 0.067785, 
idx: 200, loss: 1.502536, time: 0.080783, 
idx: 200, loss: 1.478899, time: 0.087796, 
Testing Model :skipgram, los

1. Preprocessing the data
===

In [4]:
# Change this to config file
#parameters
vocab_size = 100000
thresholds = 1e-3
embedding_size = 200
context_size = 5
architecture = 'cbow'
use_loss = 'Negative_Sampling'
sample_size = 25
lr = 1e-3
epochs = 1

In [5]:
words = preprocess('text8.zip')

In [6]:
words, word_lst, word_dict, word_reverse_dict, words_index, unigram_dict = total_sampling(file_data=words, subsampling=True, threshold=thresholds)

making dictionary :  3.089906692504883
unknown count :  4.180799961090088
unigram dictionary :  0.05385613441467285
subsampling time :  10.712352275848389


In [7]:
print (word_reverse_dict[1], unigram_dict[1])

the 0.06241594118789615


2. negative Sampling and Hierarchical Softmax
====

In [8]:
sample_table = Unigram_Table(vocab_size,sample_size,unigram_dict)
huffman_encode = Huffman_encoding(unigram_dict)
huffman_encode.encoding()
node_lst, node_dict, reverse_node_dict = huffman_encode.make_node_dict()

Filling Unigram Table


In [9]:
architectures = ['cbow', 'skipgram']
use_loss = ['Negative_Sampling', 'Hierarchical_Softmax']
learning_rate = {'Softmax':1e-5, 'Negative_Sampling':0.025, 'Hierarchical_Softmax':0.025}

In [10]:
for architecture in architectures:
    for loss_type in use_loss:
        print ('Testing Model :%s, loss : %s' % (architecture, loss_type))
        model = Word2Vec(vocab_size, embedding_size, context_size, architecture=architecture, verbose=len(words_index)//50,
                        use_loss=loss_type, learning_rate = learning_rate[loss_type], sample_table=sample_table, encoder=node_dict,
                        decay_step=len(words_index)//20, decay_rate=0.98, neg_samples=sample_size, epochs=epochs)
        model.train(words_index)
        #print (learning_rate[loss_type])
        #learning_rate['Negative_Sampling'] = learning_rate[loss_type]
    # model save
np.save('./results/%d_%s_%s_input.npy' % (e, architecture, loss_type), model.params['inputVectors']) # 0.15
np.save('./results/%d_%s_%s_output.npy'% (e, architecture, loss_type), model.params['outputVectors']) # 0.15 

Testing Model :cbow, loss : Negative_Sampling
idx: 237507, loss: 5.692648, time: 68.845177, 


KeyboardInterrupt: 